## Limpieza y validación de las noticias
<p>En este script se consolidan los df de las noticias extraídas, se limpian algunas columnas y se presenta el dataframe final para el procesamiento semántico</p>

In [1]:
import pandas as pd 

# Cargo e inspecciono los conjuntos de datos 
data_eldiario = pd.read_csv('../data/raw/eldiario_politica_economia_completo.csv')
data_analisis = pd.read_csv('../data/raw/analisis_provinciales_economia_completo.csv')
data_heraldo = pd.read_csv('../data/raw/noticias_heraldo_politica_economia.csv')

# Inspecciono los datos
print("Datos de El Diario:", data_eldiario.columns)
print("Datos de Análisis:", data_analisis.columns)
print("Datos de El Heraldo:", data_heraldo.columns)


Datos de El Diario: Index(['enlace', 'seccion', 'pagina', 'fecha_publicacion', 'titulo',
       'descripcion', 'vistas', 'categorias'],
      dtype='object')
Datos de Análisis: Index(['seccion', 'pagina', 'enlace', 'titulo', 'fecha', 'contenido'], dtype='object')
Datos de El Heraldo: Index(['seccion', 'pagina', 'enlace', 'titulo', 'fecha', 'copete',
       'contenido'],
      dtype='object')


In [2]:
# Creamos columna con nombre del medio
data_eldiario['medio'] = 'eldiario'
data_analisis['medio'] = 'analisis'
data_heraldo['medio'] = 'elheraldo'

# Estandarizar columnas
data_eldiario.rename(columns={'titulo': 'titulo', 'enlace': 'enlace', 'fecha_publicacion': 'fecha', 'descripcion': 'contenido'}, inplace=True)

# Tratamiento de fechas
print('eldiario', data_eldiario['fecha'].head(3))
MESES = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
    "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}
fechas_eldiario = []
for f in data_eldiario['fecha']:
    if isinstance(f, str) and ',' in f:
        partes = f.replace(',', '').lower().split()
        dia = int(partes[0])
        mes = MESES[partes[1]]
        anio = int(partes[2])
        fechas_eldiario.append(pd.Timestamp(year=anio, month=mes, day=dia))
    else:
        fechas_eldiario.append(pd.NaT)
data_eldiario['fecha'] = fechas_eldiario
print(data_eldiario['fecha'].head(3))

# Análisis: fecha tipo '20 de Julio de 2025 - 11:46'
print('analisis:', data_analisis['fecha'].head(3))
fechas_analisis = []
for f in data_analisis['fecha']:
    if isinstance(f, str) and 'de' in f and '-' in f:
        partes = f.lower().split(' - ')
        fecha_partes = partes[0].replace('de ', '').split()
        dia = int(fecha_partes[0])
        mes = MESES[fecha_partes[1]]
        anio = int(fecha_partes[2])
        hora, minuto = map(int, partes[1].split(':'))
        fechas_analisis.append(pd.Timestamp(year=anio, month=mes, day=dia, hour=hora, minute=minuto))
    else:
        fechas_analisis.append(pd.NaT)
data_analisis['fecha'] = fechas_analisis
print(data_analisis['fecha'].head(3))

# El Heraldo: fecha tipo '19 de julio de 2025 - 11:00'
print('elheraldo:', data_heraldo['fecha'].head(3))
fechas_heraldo = []
for f in data_heraldo['fecha']:
    if isinstance(f, str) and 'de' in f and '-' in f:
        partes = f.lower().split(' - ')
        fecha_partes = partes[0].replace('de ', '').split()
        dia = int(fecha_partes[0])
        mes = MESES[fecha_partes[1]]
        anio = int(fecha_partes[2])
        hora, minuto = map(int, partes[1].split(':'))
        fechas_heraldo.append(pd.Timestamp(year=anio, month=mes, day=dia, hour=hora, minute=minuto))
    else:
        fechas_heraldo.append(pd.NaT)
data_heraldo['fecha'] = fechas_heraldo
print(data_heraldo['fecha'].head(3))



eldiario 0    18 julio, 2025
1    18 julio, 2025
2    18 julio, 2025
Name: fecha, dtype: object
0   2025-07-18
1   2025-07-18
2   2025-07-18
Name: fecha, dtype: datetime64[ns]
analisis: 0    20 de Julio de 2025 - 11:46
1    19 de Julio de 2025 - 16:37
2    19 de Julio de 2025 - 09:41
Name: fecha, dtype: object
0   2025-07-20 11:46:00
1   2025-07-19 16:37:00
2   2025-07-19 09:41:00
Name: fecha, dtype: datetime64[ns]
elheraldo: 0    19 de julio de 2025 - 11:00
1    19 de julio de 2025 - 06:30
2    18 de julio de 2025 - 22:45
Name: fecha, dtype: object
0   2025-07-19 11:00:00
1   2025-07-19 06:30:00
2   2025-07-18 22:45:00
Name: fecha, dtype: datetime64[ns]


In [3]:
# Unimos los DataFrames 
columnas_finales = ['titulo', 'descripcion', 'fecha', 'enlace', 'medio']
for df in [data_eldiario, data_analisis, data_heraldo]:
    for col in columnas_finales:
        if col not in df.columns:
            df[col] = ""
df_noticias = pd.concat([data_eldiario, data_analisis, data_heraldo], ignore_index=True)

# Unificar los textos
df_noticias["titulo"] = df_noticias["titulo"].fillna("")
df_noticias["descripcion"] = df_noticias["descripcion"].fillna("")
df_noticias["texto_completo"] = df_noticias["titulo"] + ". " + df_noticias["descripcion"]

# Limpieza básica
import re
import unicodedata

def limpiar_texto(texto):
    if pd.isnull(texto):
        return ""
    texto = texto.lower()
    texto = unicodedata.normalize("NFKD", texto).encode("ascii", "ignore").decode("utf-8")
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'[^a-zA-Z0-9áéíóúñü ]', '', texto)  # Solo letras, números y espacios
    return texto.strip()

df_noticias["texto_limpio"] = df_noticias["texto_completo"].apply(limpiar_texto)


In [4]:
df_noticias.head()
df_noticias.to_csv('../data/processed/noticias_limpias.csv', index=False)
print("Datos limpios guardados en 'noticias_limpias.csv'")
# --- IGNORE ---

Datos limpios guardados en 'noticias_limpias.csv'
